In [32]:
import numpy as np
from astropy.io import fits
import pandas as pd

In [66]:
#Read CSC 2.1 into dataframe
filename='/Users/kciurleo/Documents/kciurleo/AGN/unorganized/CSC2.1p_OIR_SDSSspecmatch.csv'
data=pd.read_csv(filename)
columns = data.columns

/var/folders/6x/hz5y30457mqg1y9dl2fg7r5m00034d/T/ipykernel_34251/3238967991.py:3: DtypeWarning: Columns (57,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(filename)


In [67]:
#Find only sources with SDSS data
sources = data.dropna(subset=['Sep_SPEC_CSC21P'])
print(f'All source count: {len(sources)}, {len(data)-len(sources)} not observed with SDSS')

#Find only non-extended CSC sources
point_sources = sources.loc[sources['extent_flag']==False]
print(f'Point source count: {len(point_sources)}, removed {len(sources)-len(point_sources)}')

All source count:17666, 369775 not observed with SDSS
Point source count:16022, removed 1644


In [68]:
#Saved point sources as csv, which was used in SciServer CasJobs SQL query to get Portsmouth classifications
point_sources.to_csv('/Users/kciurleo/Documents/kciurleo/AGN/unorganized/point_sources.csv', index=False) 

In [99]:
#Portsmouth classifications https://salims.pages.iu.edu/agn/
portsmouth=pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/unorganized/point_source_classifications.csv')

#Agostino classifications https://salims.pages.iu.edu/agn/
agostino=pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/unorganized/agostino2021_table1.csv')

In [119]:
#Get the agostino spectral ids and merge into normal agostino table
agostino_IDs=pd.read_csv('/Users/kciurleo/Documents/kciurleo/AGN/unorganized/agostino_specIDs.csv')
agostino_full =pd.merge(agostino_IDs, agostino, left_on=['objID'], right_on=['SDSS_ObjID'], how='inner')

#Combine our point source table, portsmouth classification, and agostine classifications
combined = pd.merge(point_sources, portsmouth, left_on=['PLATE', 'MJD', 'FIBERID'],right_on=['plate', 'mjd', 'fiberID'], how='left')
classified_point_sources = pd.merge(combined, agostino_full, left_on =['specobjID'],right_on=['specobjID'],how='left')

In [132]:
#Find only Seyfert Galaxies, classified as bpt="Seyfert" for Portsmouth 
portsmouth_s2=classified_point_sources.loc[classified_point_sources['bpt']=="Seyfert"]
print(f'Portsmouth Seyferts:{len(portsmouth_s2)}, {len(point_sources)-len(portsmouth_s2)} non-Seyferts')

#and sl_class1=1 for Agostino
agostino_s2=classified_point_sources.loc[classified_point_sources['sl_class1']==1]
print(f'Agostino Seyferts: {len(agostino_s2)}, {len(point_sources)-len(agostino_s2)} non-Seyferts')

#Those classified by both
inner_s2=pd.merge(agostino_s2, portsmouth_s2, how='inner')
print(f'Portsmouth-Agostino Seyferts: {len(inner_s2)}')

#Those classified by either
outer_s2=pd.merge(agostino_s2, portsmouth_s2, how='outer')
print(f'Portsmouth or Agostino Seyferts: {len(outer_s2)}')

Portsmouth Seyferts:660, 15362 non-Seyferts
Agostino Seyferts:130, 15892 non-Seyferts
Portsmouth-Agostino Seyferts:73
Portsmouth or Agostino Seyferts:717
